# Microsoft Recently developed a gradient boosting machine which will be tested on the 19h_44d dataset.

The wrapper for python can be found at:

https://github.com/ArdalanM/pyLightGBM/blob/master/examples/binary_classification.py

Note that path_to_exec needs to be specified and is the file path to the lightgbm installer.

A quickstart guide with paramter list can be found at: 

https://github.com/ArdalanM/pyLightGBM

Note that referencing path_to_exec has different formatting for different OS. 

A reference for how the parameters compare to XGBoost and small tutorial range of values see:
https://github.com/Microsoft/LightGBM/blob/master/docs/Parameters.md

In [2]:
from pylightgbm.models import GBMClassifier
import numpy as np
import pandas as pd
from sklearn import datasets, metrics, model_selection
from sklearn import metrics
from sklearn.model_selection import cross_val_predict

path_to_exec = "/home/tom/IdeaProjects/Astronomy Supervised Project/LightGBM/build/build/LightGBM/build/LightGBM/lightgbm"


df = pd.read_csv("TESSfield_19h_44d_combinedfeatures_try2.csv", index_col=0)
X = df.drop(['Ids', 'CatalogY', 'ManuleY', 'CombinedY', 'Catalog_Period',
             'Depth', 'Catalog_Epoch', 'SNR'], axis=1)

y = df['CombinedY']

def modelfit(alg, X, y, cv_folds=4):
    # StratifiedKFold automatically used by cross_val_predict on binary classification
    # bear in mind that this does not use trapezfoid rule
    # y_pred calculates the probabilities that each value is 1 or 0 using stratified cross validation
    # pr_auc calculates the area under a precision recall curve
    y_pred = cross_val_predict(alg, X, y, cv=cv_folds, 
                               method='predict_proba')[:,1]
    pr_auc = metrics.average_precision_score(y, y_pred)
    auc_roc = metrics.roc_auc_score(y, y_pred)
    print 'The pr and roc scores are: '
    return pr_auc, auc_roc


params = {'exec_path': path_to_exec, 'verbose': False}


clf = GBMClassifier(application='binary', **params)


modelfit(clf, X, y)

The pr and roc scores are: 


(0.76458332695049502, 0.90470449476675463)

We see that the default scores are quite high practically on par with the XGBoost eqivalent and it runs more quickly, only taking ~20 seconds on my laptop. 

First we can try using the optimal XGBoost values for the same dataset.  

In [3]:
params = {'exec_path': path_to_exec, 'verbose': False,
          'num_iterations':100, 'learning_rate':.1, 'num_leaves':1024,
          'min_data_in_leaf':1}

# binary tells GBM that its binary classification
clf = GBMClassifier(application='binary', **params)

modelfit(clf, X, y)

The pr and roc scores are: 


(0.76152641708855184, 0.89857018484999152)

Some notes about the parameters: 

Feature_fraction:
Selects part of features on each iteration, so if it is .80 it will select 80 percent of the features before training each tree. 

In [4]:
from sklearn.grid_search import GridSearchCV

param_test1 = {'num_leaves':[1, 10, 100, 1000], 'min_data_in_leaf':
               [1, 10, 100, 1000]}


gsearch = GridSearchCV(estimator = GBMClassifier(application='binary',
                                                 exec_path=path_to_exec,
                                                 verbose= False, 
                                                 num_threads=4), 
                       param_grid = param_test1, 
                       scoring='roc_auc',
                       n_jobs=4,
                       iid=False,
                       cv=4)

gsearch.fit(X, y)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

JoblibCalledProcessError: JoblibCalledProcessError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/home/tom/anaconda3/envs/py27/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    169     pkg_name = mod_name.rpartition('.')[0]
    170     main_globals = sys.modules["__main__"].__dict__
    171     if alter_argv:
    172         sys.argv[0] = fname
    173     return _run_code(code, main_globals, None,
--> 174                      "__main__", fname, loader, pkg_name)
        fname = '/home/tom/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = 'ipykernel'
    175 
    176 def run_module(mod_name, init_globals=None,
    177                run_name=None, alter_sys=False):
    178     """Execute a module's code without importing it

.